In [1]:
import os 
import requests
from bs4 import BeautifulSoup
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

import re


/Users/charanmannuru/miniconda3/envs/VA/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.1.19) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
my_activeloop_org_id = "charanvardhan"
my_activeloop_dataset_name = "VoiceAssistant-embeddings"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [4]:
def get_documents_url(url):
    return [
    '/docs/huggingface_hub/guides/overview',
    '/docs/huggingface_hub/guides/download',
    '/docs/huggingface_hub/guides/upload',
    '/docs/huggingface_hub/guides/hf_file_system',
    '/docs/huggingface_hub/guides/repository',
    '/docs/huggingface_hub/guides/search',
    ]

def construct_url(base_url, relative_paath):
    return base_url + relative_paath

In [5]:
def scrape_page_content(url):

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Extract the desired content from the page 
    text = soup.body.text.strip()

    # remove none ascii characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

def scrape_all_content(base_url, relative_paths, 
                       filename):
    all_text = []
    for path in relative_paths:
        url = construct_url(base_url, path)
        text = scrape_page_content(url)
        all_text.append(text)
    

    with open(filename, 'w', encoding='utf-8') as f:
        for item in all_text:
            f.write(item + '\n')
        
    return all_text

In [6]:
# load and splitting the text
# def load_docs(root_dir, filename):
#     docs = []
#     try:
#         loader = TextLoader(os.path.join(root_dir, filename, encoding='utf-8'))
#         docs.extend(loader.load_and_split())
    
#     except Exception as e:
#         pass
    
#     return docs

def load_docs(root_dir, filename):
    docs = []
    try:
        file_path = os.path.join(root_dir, filename)
        loader = TextLoader(file_path, encoding='utf-8')  # ✅ fixed encoding
        docs = loader.load()  # ✅ use .load()
    except Exception as e:
        print(f"Error loading documents: {e}")  # ✅ helpful error message
    return docs

def split_docs(docs):
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    return text_splitter.split_documents(docs)

In [ ]:
def main():
    base_url = 'https://huggingface.co'

    filename = 'content.txt'
    root_dir = './'
    relative_paths = get_documents_url(base_url)
    content = scrape_all_content(base_url, relative_paths, filename)
    docs = load_docs(root_dir, filename)

    texts = split_docs(docs)

    # db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)
    # db.add_documents(texts)
    os.remove(filename)
    

In [ ]:
if __name__ == "__main__":
    main()

In [11]:
import openai
import streamlit as st
from audio_recorder_streamlit import audio_recorder
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from elevenlabs import generate
from streamlit_chat import message

In [12]:
TempAudioFile = "temp.wav"
audioformat = "audio/wav"



In [13]:
def load_embeddings():
    embeddings = OpenAIEmbeddings()
    db = DeepLake(dataset_path=dataset_path, read_only=True, embedding_function=embeddings)

    return db



In [14]:
def transcribe_audio_to_text(audio_file):
    try:
        with open(audio_file, "rb") as f:
            response = openai.Audio.transcribe(
                model="whisper-1",
                file=f,
                response_format="text"
            )

        return response["text"]
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None


In [19]:
def record_and_transcribe():
    audioBytes = audio_recorder()
    transcripts = None
    if audioBytes:
        st.audio(audioBytes, format=audioformat)
        
        with open(TempAudioFile, "wb") as f:
            f.write(audioBytes)

        if st.button("Transcribe"):
            transcription = transcribe_audio_to_text(TempAudioFile)
            os.remove(TempAudioFile)
            display_transcription(transcription)

    return transcription

In [20]:
def display_transcription(transcription):
    if transcription:
        st.write(f"transaction: {transcription}")
        with open("audioTranscriopt.txt", "w+") as f:
            f.write(transcription)
    else:
        st.write("error transcribing audio")


def get_user_text(transcription):
    return st.text_input("", value=transcription if transcription else "", key=input)

